In [12]:
from datetime import datetime
import backtrader as bt
# 导入pyfolio 包
import pyfolio as pf


# 创建策略类
class SmaCross(bt.Strategy):
    # 定义参数
    params = dict(period=5)  # 移动平均期数

    # 日志函数
    def log(self, txt, dt=None):
        '''日志函数'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # 订单状态 submitted/accepted，无动作
            return
        # 订单完成
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('买单执行, %.2f' % order.executed.price)
            elif order.issell():
                self.log('卖单执行, %.2f' % order.executed.price)
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('订单 Canceled/Margin/Rejected')

    # 记录交易收益情况（可省略，默认不输出结果）
    def notify_trade(self, trade):
        if trade.isclosed:
            print('毛收益 %0.2f, 扣佣后收益 % 0.2f, 佣金 %.2f' %
                  (trade.pnl, trade.pnlcomm, trade.commission))

    def __init__(self):
        # 移动平均线指标
        self.move_average = bt.ind.MovingAverageSimple(
            self.data, period=self.params.period)
        # 交叉信号指标
        self.crossover = bt.ind.CrossOver(self.data, self.move_average)

    def next(self):
        if not self.position:  # 还没有仓位
            # 当日收盘价上穿5日均线，创建买单，买入100股
            if self.crossover > 0:
                self.log('创建买单')
                self.buy(size=100)
        # 有仓位，并且当日收盘价下破5日均线，创建卖单，卖出100股
        elif self.crossover < 0:
            self.log('创建卖单')
            self.sell(size=100)


##########################
# 主程序开始
#########################

# 创建大脑引擎对象
cerebro = bt.Cerebro()


# 数据文件路径
datapath = '../Data/600466.SH-day.csv'

# 创建行情数据对象，加载数据
data = bt.feeds.GenericCSVData(
    dataname=datapath,
    datetime=0,  # 日期行所在列
    open=1,  # 开盘价所在列
    high=2,  # 最高价所在列
    low=3,  # 最低价所在列
    close=4,  # 收盘价价所在列
    volume=5,  # 成交量所在列
    openinterest=-1,  # 无未平仓量列
    dtformat='%Y-%m-%d',  # 日期格式
    fromdate=datetime(2019, 1, 1),  # 起始日
    todate=datetime(2021, 1, 1))  # 结束日

cerebro.adddata(data)  # 将行情数据对象注入引擎
cerebro.addstrategy(SmaCross)  # 将策略注入引擎

cerebro.broker.setcash(10000.0)  # 设置初始资金

# 加入pyfolio分析者
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')

results=cerebro.run()  # 运行
strat = results[0]
pyfoliozer = strat.analyzers.getbyname('pyfolio')
returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()
pf.create_full_tear_sheet(returns)

2019-01-18, 创建买单
2019-01-21, 买单执行, 33.06
2019-01-21, 创建卖单
2019-01-22, 卖单执行, 32.39
毛收益 -67.00, 扣佣后收益 -67.00, 佣金 0.00
2019-01-23, 创建买单
2019-01-24, 买单执行, 32.51
2019-01-24, 创建卖单
2019-01-25, 卖单执行, 32.94
毛收益 43.00, 扣佣后收益  43.00, 佣金 0.00
2019-01-25, 创建买单
2019-01-28, 买单执行, 33.31
2019-02-15, 创建卖单
2019-02-18, 卖单执行, 34.98
毛收益 167.00, 扣佣后收益  167.00, 佣金 0.00
2019-02-18, 创建买单
2019-02-19, 买单执行, 35.96
2019-02-19, 创建卖单
2019-02-20, 卖单执行, 35.16
毛收益 -80.00, 扣佣后收益 -80.00, 佣金 0.00
2019-02-25, 创建买单
2019-02-26, 买单执行, 37.07
2019-03-08, 创建卖单
2019-03-11, 卖单执行, 38.37
毛收益 130.00, 扣佣后收益  130.00, 佣金 0.00
2019-03-12, 创建买单
2019-03-13, 买单执行, 40.59
2019-03-26, 创建卖单
2019-03-27, 卖单执行, 44.85
毛收益 426.00, 扣佣后收益  426.00, 佣金 0.00
2019-03-27, 创建买单
2019-03-28, 买单执行, 45.22
2019-03-28, 创建卖单
2019-03-29, 卖单执行, 43.55
毛收益 -167.00, 扣佣后收益 -167.00, 佣金 0.00
2019-03-29, 创建买单
2019-04-01, 买单执行, 46.14
2019-04-08, 创建卖单
2019-04-09, 卖单执行, 47.93
毛收益 179.00, 扣佣后收益  179.00, 佣金 0.00
2019-04-09, 创建买单
2019-04-10, 买单执行, 51.20
2019-04-11, 创建卖单
2019-04-1

C:\ProgramData\Anaconda3\envs\py39\lib\site-packages\empyrical\utils.py:440: UserWarning: Yahoo Finance read failed: mktime argument out of range, falling back to Google
  warnings.warn(


AttributeError: module 'pandas_datareader.data' has no attribute 'get_data_google'

In [15]:
import datetime
import backtrader as bt
import backtrader.analyzers as btanalyzers
import backtrader.feeds as btfeeds
import backtrader.strategies as btstrats

cerebro = bt.Cerebro()

dataname = '../data/2005-2006-day-001.txt'
data = btfeeds.BacktraderCSVData(dataname=dataname)
cerebro.adddata(data)
cerebro.addstrategy(btstrats.SMA_CrossOver)

cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
results = cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

strat = results[0]
pyfoliozer = strat.analyzers.getbyname('pyfolio')
returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()
import pyfolio as pf
pf.create_full_tear_sheet(
    returns,
    positions=positions,
    transactions=transactions,
    live_start_date='2005-05-01',
    )

cerebro.plot()

Starting Portfolio Value: 10000.00
Final Portfolio Value: 10496.68


C:\ProgramData\Anaconda3\envs\py39\lib\site-packages\empyrical\utils.py:440: UserWarning: Yahoo Finance read failed: mktime argument out of range, falling back to Google
  warnings.warn(


AttributeError: module 'pandas_datareader.data' has no attribute 'get_data_google'

In [18]:
import datetime

import backtrader as bt
import backtrader.analyzers as btanalyzers
import backtrader.feeds as btfeeds
import backtrader.strategies as btstrats

cerebro = bt.Cerebro()

dataname = '../data/2005-2006-day-001.txt'
data = btfeeds.BacktraderCSVData(dataname=dataname)
cerebro.adddata(data)
cerebro.addstrategy(btstrats.SMA_CrossOver)


#……策略和参数
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='PyFolio')
back = cerebro.run()
strat = back[0]
portfolio_stats = strat.analyzers.getbyname('PyFolio')
returns, positions, transactions, gross_lev = portfolio_stats.get_pf_items()
print(returns)
returns.index = returns.index.tz_convert(None)
import quantstats
quantstats.reports.html(returns, output='stats.html', title='BTC Sentiment')



index
2005-01-03 00:00:00+00:00    0.000000
2005-01-04 00:00:00+00:00    0.000000
2005-01-05 00:00:00+00:00    0.000000
2005-01-06 00:00:00+00:00    0.000000
2005-01-07 00:00:00+00:00    0.000000
                               ...   
2006-12-21 00:00:00+00:00   -0.000614
2006-12-22 00:00:00+00:00   -0.003680
2006-12-27 00:00:00+00:00    0.005872
2006-12-28 00:00:00+00:00   -0.000400
2006-12-29 00:00:00+00:00   -0.001020
Name: return, Length: 512, dtype: float64
